Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
#problem 1
hidden_size=1024
batch_size =128

beta1=0.001
beta2=0.001

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [7]:
num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
initialized
Minibatch loss at step 0: 724.450012
Minibatch accuracy: 14.8%
Validation accuracy: 18.3%
Minibatch loss at step 500: 299.790192
Minibatch accuracy: 78.1%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 264.927216
Minibatch accuracy: 80.5%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 240.588593
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 211.229706
Minibatch accuracy: 87.5%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 191.567795
Minibatch accuracy: 89.1%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 172.412094
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Test accuracy: 90.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
#problem 2
    
train_subset=train_dataset[:3000,:]
train_labels_subset=train_labels[:3000,:]

hidden_size=1024
batch_size =10

beta1=0.001
beta2=0.001

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [19]:
num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels_subset.shape[0]-batch_size)
        
        batch_data=train_subset[offset:(offset+batch_size),:]
        batch_labels=train_labels_subset[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


initialized
Minibatch loss at step 0: 648.305542
Minibatch accuracy: 0.0%
Validation accuracy: 26.0%
Minibatch loss at step 500: 402.247620
Minibatch accuracy: 70.0%
Validation accuracy: 76.4%
Minibatch loss at step 1000: 258.451691
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 235.627960
Minibatch accuracy: 90.0%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 217.453583
Minibatch accuracy: 90.0%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 192.702484
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 174.529831
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Test accuracy: 88.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [20]:
#Problem 3
#Dropouts

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    hidden_1=tf.nn.dropout(hidden_1,0.5)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [21]:
num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels_subset.shape[0]-batch_size)
        
        batch_data=train_subset[offset:(offset+batch_size),:]
        batch_labels=train_labels_subset[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


initialized
Minibatch loss at step 0: 909.068359
Minibatch accuracy: 10.0%
Validation accuracy: 26.2%
Minibatch loss at step 500: 802.409119
Minibatch accuracy: 60.0%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 293.231537
Minibatch accuracy: 90.0%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 330.692322
Minibatch accuracy: 80.0%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 279.105774
Minibatch accuracy: 90.0%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 259.228668
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 275.079773
Minibatch accuracy: 90.0%
Validation accuracy: 79.4%
Test accuracy: 86.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [22]:
#Problem 5
hidden_size=1024
hidden2_size=300

batch_size=128

beta1=0.01
beta2=0.01
beta3=0.0001

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
#     hidden_1=tf.nn.dropout(hidden_1,0.5)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, hidden2_size]))
    biases_2 = tf.Variable(tf.zeros([hidden2_size]))
    
    hidden_2=tf.nn.relu(tf.matmul(hidden_1,weights_2)+biases_2)
    
#     hidden_2=tf.nn.dropout(hidden_2,0.5)
  
    weights_3 = tf.Variable(
    tf.truncated_normal([hidden2_size, num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_2,weights_3)+biases_3
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    +beta3*tf.nn.l2_loss(weights_3)

    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.04, global_step,1000,0.90,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
#     optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)

In [23]:
num_steps=9001

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_dataset.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            
        global_step+=1
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


initialized
Minibatch loss at step 0: 7065.429688
Minibatch accuracy: 10.9%
Validation accuracy: 23.9%
Minibatch loss at step 1000: 1935.321533
Minibatch accuracy: 53.9%
Validation accuracy: 49.4%
Minibatch loss at step 2000: 942.141724
Minibatch accuracy: 64.8%
Validation accuracy: 66.5%
Minibatch loss at step 3000: 493.327515
Minibatch accuracy: 67.2%
Validation accuracy: 72.8%
Minibatch loss at step 4000: 275.474945
Minibatch accuracy: 78.1%
Validation accuracy: 77.5%
Minibatch loss at step 5000: 162.980484
Minibatch accuracy: 84.4%
Validation accuracy: 80.5%
Minibatch loss at step 6000: 102.091103
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Minibatch loss at step 7000: 66.779190
Minibatch accuracy: 82.8%
Validation accuracy: 82.9%
Minibatch loss at step 8000: 45.620506
Minibatch accuracy: 86.7%
Validation accuracy: 83.6%
Minibatch loss at step 9000: 32.507133
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Test accuracy: 91.1%
